In [1]:
from postbound.db import db, mysql
from postbound.qal import qal, base, clauses, transform, formatter, predicates, expressions
from postbound.optimizer import jointree
from postbound.optimizer.physops import operators as physops
from postbound.optimizer.planmeta import hints as planmeta
conn_string = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "1234",
    "database": "imdbload"}
mysql_db = mysql.connect(connection_args=mysql.MysqlConnectionArguments(**conn_string))
hint_service = mysql_db.hinting()
table1 = base.TableReference(full_name="movie_info", alias="mi")
table2 = base.TableReference("keyword", alias="ky")
table3 = base.TableReference("movie_companies", alias="mc")
table = [table1,table2,table3]
index_list = ["idx_id", "idx_t1"]
assignment  = physops.PhysicalOperatorAssignment()
plan = planmeta.PlanParameterization()
plan_parameter2 = plan.add_join_order_hint(tables=table)
join_operator1 = physops.JoinOperatorAssignment(join=[table1,table2], operator=physops.JoinOperators.BlockNestedLoopJoin)
scan_operator = physops.ScanOperatorAssignment(
    operator=physops.ScanOperators.IndexScan,
    table=table1
)
join_operator2 = physops.JoinOperatorAssignment(
    operator=physops.JoinOperators.MergeJoin,
    join=[table2, table3]
)
join_operators = {
    frozenset(join_operator1.join): join_operator1,
    frozenset(join_operator2.join): join_operator2,
}
for join, join_operator in join_operators.items():
    assignment.set_join_operator(join_operator)
# Set the scan operator in the PhysicalOperatorAssignment
assignment.set_scan_operator(scan_operator)


assignment.set_operator_enabled_globally(operator=physops.JoinOperators.BlockNestedLoopJoin, enabled=True)
tree1 = jointree.PhysicalQueryPlan.for_base_table(table=base.TableReference(full_name="movie_info", alias="mi"),base_annotation=jointree.PhysicalBaseTableMetadata(filter_predicate=predicates.BinaryPredicate(
    operation=expressions.LogicalSqlOperators.Equal,
    first_argument=base.ColumnReference(name="id",table=base.TableReference(full_name="movie_info", alias="mi")),second_argument=42),cardinality=1000))
tree2 = jointree.PhysicalQueryPlan.for_base_table(table= base.TableReference("keyword", alias="ky"),base_annotation=jointree.PhysicalBaseTableMetadata(filter_predicate=predicates.BinaryPredicate(
    operation=expressions.LogicalSqlOperators.Equal,
    first_argument=base.ColumnReference(name="id",table=base.TableReference(full_name="keyword", alias="ky")),second_argument=100),cardinality=100))
tree3 = jointree.PhysicalQueryPlan.for_base_table(table=base.TableReference("movie_companies", alias="mc"),base_annotation=jointree.PhysicalBaseTableMetadata(filter_predicate=predicates.BinaryPredicate(
    operation=expressions.LogicalSqlOperators.Equal,
    first_argument=base.ColumnReference(name="id",table=base.TableReference(full_name="movie_companies", alias="mc")),second_argument=1040),cardinality=4000))
trees = [tree1,tree2,tree3]
join_sequence = [table1,table2]
nested_sequence2 = [join_sequence, table1, table2]
join_tree = jointree.PhysicalQueryPlan.joining(left_tree=tree1,right_tree=tree2)

explain_clause = clauses.Explain.explain_analyze()
select_clause = clauses.Select(targets=[clauses.BaseProjection.column(base.ColumnReference(name="id",table=base.TableReference(full_name="movie_info", alias="mi")))])
from_clause = clauses.ImplicitFromClause(tables=table)
predicate1 = predicates.BinaryPredicate(operation=expressions.LogicalSqlOperators.Equal,first_argument=base.ColumnReference(name="id",table=base.TableReference(full_name="movie_info", alias="mi")),second_argument=base.ColumnReference(name="id",table=base.TableReference(full_name="keyword", alias="ky")))
predicate2 = predicates.BinaryPredicate(operation=expressions.LogicalSqlOperators.Equal,first_argument=base.ColumnReference(name="id",table=base.TableReference(full_name="keyword", alias="ky")),second_argument=base.ColumnReference(name="id",table=base.TableReference(full_name="movie_companies", alias="mc")))
operation = expressions.LogicalSqlCompoundOperators.And
where_clause = clauses.Where(predicates.BinaryPredicate(operation=operation,first_argument=predicate1,second_argument=predicate2))
limit_clause = clauses.Limit(limit=10)
query = qal.ImplicitSqlQuery(select_clause=select_clause,from_clause=from_clause,where_clause=where_clause,limit_clause=limit_clause)
results = hint_service.generate_hints(query=query,join_order=join_tree,physical_operators=assignment,plan_parameters=plan)
print(results)
final = mysql_db.execute_query(query=results)
print(final)


SET optimizer_switch='block_nested_loop=on';
SELECT /*+
  STRAIGHT_JOIN((mi ky))
  INDEX_SCAN(mi)
  
  BNL(mi, ky)
  MERGE(ky, mc)
  JOIN_ORDER(mi, ky, mc)
*/ mi.id FROM movie_info AS mi, keyword AS ky, movie_companies AS mc WHERE mi.id = ky.id AND ky.id = mc.id LIMIT 10;
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
